In [12]:
# Scnorr

from Crypto.Util.number import getPrime, isPrime
from hashlib import sha256

def keygen(nbit):
    while True:
        p = getPrime(nbit -1 )
        q = 2 * p + 1
        # |F_q| =  q- 1 = 2 * p
        # q = 2 * p * q + 1
        
        if isPrime(q):
            break
    G = GF(q)
    g = int(G.multiplicative_generator()**2)

    x = randint(0, p)
    return (g, q), pow(g, -x, q), x

def sign(sk, msg, g, q):
    p = q // 2
    
    k = randint(1, p-1)
    r = pow(g, k, q) # r = g^k (mod q)
    e = int.from_bytes(sha256(str(r).encode() + msg).digest()) % p
    s = (k + sk * e) % p
    return (e, s)


def verify(pk, msg, sig, g, q):
    p = q // 2
    e, s = sig

    r_v = pow(g, s, q) * pow(pk, e, q) % q
    e_v = int.from_bytes(sha256(str(r_v).encode() + msg).digest()) % p 
    return e_v == e

msg = b'mephictf'
params, pk, sk = keygen(128)
g, q = params

$r = g^k \pmod{p}, e = H(r|M), s = (k + x * e) (mod p)$

$g^s = g^{k + x * e} \pmod{q}$

$g^s = g^k * g^{x * e} \pmod{q}$

$g^s * g^{-x * e} = g^k \pmod{q}$

$g^s * (g^{-x})^e = r \pmod{q}$

$g^s * pk^e = r' \pmod{q}$

$sha256(r' | M) == e?$

In [13]:
p = q // 2
pow(g, p, q)
# pk = g^y (mod q), y \in [0, p]
M = b'mephictf'
sig = sign(sk, M, g, q)

In [10]:
p = q // 2
assert pow(g, p, q)== 1
sig

(78365350812416525990060830338136827672, 3540726230804222122937910152317247659)

In [14]:
verify(pk, msg, sig, g, q)

True

In [15]:
from Crypto.Util.number import getPrime

p = getPrime(512)
q = getPrime(512)
n = p * q
e = 0x10001

m = b'mephictf'
m = int.from_bytes(m)

c = pow(m, e, n)
c

80140791023896144829443508946841585796118200093564868060607349386481112169450823806812246033894470624331967284891879331696498719749826360446794063064709172543065417512617799152908567714454334746302065172735705015773621924028853131435960664005068547486190755165872079428967379594706529011781127270699266739618

In [17]:
d = pow(e, -1, (p - 1) * (q - 1))

m1 = pow(c, d, n)
print(m1.to_bytes(10, 'big'))

b'\x00\x00mephictf'


In [18]:
S = pow(m, d, n)
print(S)

39576787019588421874220035378038959022763664551378707979472373957562991013069632498710067689640950373646406568295752595050244096352903151232381820267407981028780768176191927056791832648055833527866565679281925009548066273060639450409111761327034065018110160313782544473960657664682033930524531842529044599964


In [19]:
m == pow(S, e, n)

True

In [22]:
m1 = 2 * 773
m2 = 5098855314701783
m1 * m2 == m

s1 = pow(m1, d, n)
s2 = pow(m2, d, n)
S3 = s1 * s2 % n
assert S3 == S

In [ ]:
E = Ellipti

In [71]:
from hashlib import blake2b

p = 21888242871839275222246405745257275088696311157297823662689037894645226208583
e = EllipticCurve(GF(p), [0, 3])
G = e.random_element()
q = e.order()
assert G.order() == e.order()
d_a = randint(1, q-1)
Q_a = d_a * G

o = G.order()
m = b"Where's your motivation?"

def sign(m):
    z = int(blake2b(m, digest_size=q.bit_length() // 16).hexdigest(), 16)
    k = randint(1, q-1)
    r = int((k * G)[0]) % o
    s = pow(int(k), -1, q) * (z + int(r) * d_a) % q
    return r, s, z


def verify(G, Q, r, s, o):
    u1 = pow(s, -1, o) * z % o
    u2 = pow(s, -1, o) * r % o

    assert int((u1 * G + u2 * Q)[0]) % o == r

r, s, z = sign(m)

verify(G, Q_a, r, s, o)

In [72]:
# https://github.com/asonnino/bls/blob/master/bls/scheme.py

from bplib.bp import BpGroup, G2Elem
from bls.utils import *


def setup():
	"""
	Generate the public parameters.

	Returns:
		- params: the publc parameters
	"""
	G = BpGroup()
	(g1, g2) = G.gen1(), G.gen2()
	(e, o) = G.pair, G.order()
	return (G, o, g1, g2, e)


def ttp_keygen(params, t, n):
	"""
	Generate keys for threshold signature (executed by a TTP).

	Parameters:
		- `params`: public parameters generated by `setup`
		- `t` (integer): the threshold parameter
		- `n` (integer): the total number of authorities

	Returns:
		- `sk` [Bn]: array containing the secret key of each authority
		- `vk` [G2Elem]: array containing the verification key of each authority
	"""
	assert n >= t and t > 0
	(G, o, g1, g2, e) = params
	# generate polynomials
	v = [o.random() for _ in range(0,t)]
	# generate shares
	sk = [poly_eval(v,i) % o for i in range(1,n+1)]
	# set keys
	vk = [xi*g2 for xi in sk]
	return (sk, vk)


def aggregate_vk(params, vks, threshold=True):
	"""
	Aggregate the verification keys.

	Parameters:
		- `params`: public parameters generated by `setup`
		- `vks` [G2Elem]: array containing the verification key of each authority
		- `threshold` (bool): optional, whether to use threshold cryptography or not

	Returns:
		- `aggr_vk` (G2Elem): aggregated verification key
	"""
	(G, o, g1, g2, e) = params
	# evaluate all lagrange basis polynomial li(0)
	filter = [vk for vk in vks if vk is not None]
	indexes = [i+1 for i, vk in enumerate(vks) if vk is not None]
	l = lagrange_basis(indexes, o) if threshold else [1 for _ in vks]
	# aggregate keys
	aggr_vk = ec_sum([l[i]*filter[i] for i in range(len(filter))])
	return aggr_vk


def sign(params, sk, m):
	"""
	Sign messages.

	Parameters:
		- `params`: public parameters generated by `setup`
		- `sk` (Bn): the secret key of the authority
		- `m` [Bn]: array containing the messages

	Returns:
		- `sigma_tilde` (G1Elem, G1Elem): blinded credential
	"""
	assert len(m) > 0
	(G, o, g1, g2, e) = params
	digest = hash(m)
	h = G.hashG1(digest)
	sigma = sk*h
	return sigma


def aggregate_sigma(params, sigs, threshold=True):
	"""
	Aggregate partial signatures.

	Parameters:
		- `params`: public parameters generated by `setup`
		- `sigs` [G1Elem]: array of partial credentials
		- `threshold` (bool): optional, whether to use threshold cryptography or not

	Returns:
		- `aggr_sigma` (G1Elem): aggregated credential
	"""
	(G, o, g1, g2, e) = params
	# evaluate all lagrange basis polynomial li(0)
	filter = [sig for sig in sigs if sig is not None]
	indexes = [i+1 for i, sig in enumerate(sigs) if sig is not None]
	l = lagrange_basis(indexes, o) if threshold else [1 for _ in sigs]
	# aggregate sigature
	aggr_s = ec_sum([l[i]*filter[i] for i in range(len(filter))])
	return aggr_s


def verify(params, aggr_vk, sigma, m):
	"""
	Verify signature.

	Parameters:
		- `params`: public parameters generated by `setup`
		- `aggr_vk` (G2Elem): aggregated verification key
		- `sigma` (G1Elem): signature
		- `m` [Bn]: array containing the messages

	Returns:
		- `ret` (bool): whether the credential verifies
	"""
	(G, o, g1, g2, e) = params
	digest = hash(m)
	h = G.hashG1(digest)
	return not h.isinf() and e(sigma, g2) == e(h, aggr_vk)

ModuleNotFoundError: No module named 'bplib'